In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import ollama
from src.models import Models, show_models, load_model

show_models()

model = load_model("llama_3_2_1b")

['llama_3_2_1b', 'llama_3_2_3b', 'llama_3_1_8b', 'deepseek_qwen_32b']


In [3]:
info = ollama.show(model).modelinfo
for k, v in info.items():
    print(f"{k}: {v}")

general.architecture: llama
general.basename: Llama-3.2
general.file_type: 18
general.finetune: Instruct
general.languages: ['en', 'de', 'fr', 'it', 'pt', 'hi', 'es', 'th']
general.license: llama3.2
general.parameter_count: 1235814432
general.quantization_version: 2
general.size_label: 1B
general.tags: ['facebook', 'meta', 'pytorch', 'llama', 'llama-3', 'text-generation']
general.type: model
llama.attention.head_count: 32
llama.attention.head_count_kv: 8
llama.attention.key_length: 64
llama.attention.layer_norm_rms_epsilon: 1e-05
llama.attention.value_length: 64
llama.block_count: 16
llama.context_length: 131072
llama.embedding_length: 2048
llama.feed_forward_length: 8192
llama.rope.dimension_count: 64
llama.rope.freq_base: 500000
llama.vocab_size: 128256
quantize.imatrix.chunks_count: 125
quantize.imatrix.dataset: /training_dir/calibration_datav3.txt
quantize.imatrix.entries_count: 112
quantize.imatrix.file: /models_out/Llama-3.2-1B-Instruct-GGUF/Llama-3.2-1B-Instruct.imatrix
tokenize

In [4]:
from ollama import chat

prompt = "Give me a simple recipe for a delicious citrusy cake. Make sure units are in grams when it makes sense. Temperatures should be in C."

response = chat(
    model=model,
    messages=[
        {"role": "user", "content": prompt},
    ],
)
print(response.message.content)

Here's a simple recipe for a refreshing citrusy cake:

**Lemon Poppyseed Cake**

**Servings:** 8-10
**Prep Time:** 20 minutes
**Cook Time:** 35-40 minutes
**Total Time:** 55-60 minutes

**Ingredients:**

* 250g all-purpose flour
* 100g granulated sugar
* 50g unsalted butter, softened
* 2 large eggs
* 200g fresh lemon juice (about 4 lemons)
* 1 tsp baking powder
* 0.5 tsp salt
* 2 tbsp freshly grated lemon zest
* 2 tbsp poppy seeds
* Confectioners' sugar for dusting (optional)

**Instructions:**

1. Preheat your oven to 180°C (350°F) and grease two 20cm (8-inch) round cake pans.
2. In a medium bowl, whisk together flour, sugar, baking powder, and salt.
3. In a large bowl, use an electric mixer to beat the butter until creamy. Add eggs one at a time, beating well after each addition.
4. Gradually add the dry ingredients to the wet ingredients, alternating with lemon juice, beginning and ending with the dry ingredients. Beat just until combined.
5. Stir in lemon zest and poppy seeds.
6. D

In [5]:
from ollama import ChatResponse, chat
from pydantic.types import JsonSchemaValue
from typing import Optional, List

system_prompt = "You are a helpful assistant that provides clear and concise answers to the user's needs. Always answer in a JSON format."

def generate(
    prompt: str,
    json_format: Optional[JsonSchemaValue] = None,
    model=model,
    system_prompt=system_prompt,
) -> str:
    response: ChatResponse = chat(
        model=model,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": prompt},
        ],
        options={
            "num_ctx": 4096,
            "num_predict": 1024,
            "top_k": 50,
            "top_p": 0.95,
            "temperature": 0.0,
            "seed": 0,  # this is not needed when temp is 0
            "repeat_penalty": 1.0,  # remain default for json outputs, from experience.
        },
        format=json_format,
        stream=False,
    )
    return response.message.content

# prompt = "give me 5 interesting facts about the universe"
prompt = "Give me a simple recipe for a delicious citrusy cake. Make sure units are in grams when it makes sense. Temperatures should be in C."
print(generate(prompt))

{
  "recipe": {
    "name": "Citrusy Cake",
    "ingredients": [
      {
        "name": "Flour",
        "quantity": 250g,
        "unit": "grams"
      },
      {
        "name": "Sugar",
        "quantity": 200g,
        "unit": "grams"
      },
      {
        "name": "Baking powder",
        "quantity": 5g,
        "unit": "grams"
      },
      {
        "name": "Salt",
        "quantity": 2g,
        "unit": "grams"
      },
      {
        "name": "Butter",
        "quantity": 100g,
        "unit": "grams"
      },
      {
        "name": "Eggs",
        "quantity": 4,
        "unit": "units"
      },
      {
        "name": "Milk",
        "quantity": 250g,
        "unit": "grams"
      },
      {
        "name": "Zest of 1 orange",
        "quantity": 20g,
        "unit": "grams"
      },
      {
        "name": "Zest of 1 lemon",
        "quantity": 15g,
        "unit": "grams"
      },
      {
        "name": "Juice of 1 lemon",
        "quantity": 30g,
        "unit": "gra

In [6]:
from pydantic import BaseModel

class Ingredient(BaseModel):
    name: str
    quantity: float
    unit: str

class RecipeInstruction(BaseModel):
    step: int
    description: str

class Recipe(BaseModel):
    title: str
    ingredients: List[Ingredient]
    instructions: List[RecipeInstruction]
    tools: List[str]

# we can now use eval to properly format the json as an object
# using `eval`from the output of an API is generally not safe, but we can safely do it from the JSON-output of a local model.
eval(generate(prompt, json_format=Recipe.model_json_schema()))

{'title': 'Citrusy Cake Recipe',
 'ingredients': [{'name': 'All-purpose flour',
   'quantity': 250,
   'unit': 'grams'},
  {'name': 'Granulated sugar', 'quantity': 200, 'unit': 'grams'},
  {'name': 'Unsalted butter, softened', 'quantity': 150, 'unit': 'grams'},
  {'name': 'Egg, large', 'quantity': 2, 'unit': 'grams'},
  {'name': 'Zest of 1 lemon', 'quantity': 10, 'unit': 'grams'},
  {'name': 'Zest of 1 orange', 'quantity': 10, 'unit': 'grams'},
  {'name': 'Zest of 1 lime', 'quantity': 10, 'unit': 'grams'},
  {'name': 'Vanilla extract', 'quantity': 5, 'unit': 'grams'},
  {'name': 'Citrus juice (e.g. lemon, orange, lime)',
   'quantity': 50,
   'unit': 'grams'}],
 'instructions': [{'step': 1,
   'description': 'Preheat the oven to 180°C (350°F). Grease two 20cm (8 inch) round cake pans and line the bottoms with parchment paper.'},
  {'step': 2,
   'description': 'In a medium bowl, whisk together flour, sugar, and salt.'},
  {'step': 3,
   'description': 'In a large bowl, using an electri